In [1]:
import time
import re
from collections import defaultdict

In [2]:
class CSVReader:
  def __init__(self, path, delimiter=','):
    self.reader = open(path, 'r', encoding='utf-8')
    self.delimiter = delimiter

  def readCell(self):
    cell = []
    inQuotes = False

    while True:
      c = self.reader.read(1)

      if not c:
        break

      if not inQuotes and c == self.delimiter:
        break

      if c == '"':
        inQuotes = not inQuotes

      cell.append(c)

    return ''.join(cell)

  def skipLine(self):
    self.reader.readline()

  def readIDContent(self):
    id = self.readCell()

    if not id:
      return None, None

    id = int(id)

    for _ in range(6):
      self.readCell()

    content = self.readCell()
    self.skipLine()

    return id, content

In [3]:
class InvertedIndex:
  def __init__(self):
    self.termDict = defaultdict(set)
    self.totalDocs = 0
    self.totalTerm = 0

  def addToDict(self, id, word):
    self.termDict[word].add(id)

  def tokenize(self, content):
    return re.findall(r'(|\w+)', content.lower())

  def build(self, reader):
    reader.skipLine()

    while True:
      id, content = reader.readIDContent()

      if id is None:
        print("Finished building index")
        break

      self.totalDocs = id + 1
      words = self.tokenize(content)
      for word in words:
        self.addToDict(id, word)

    self.totalTerm = len(self.termDict)


In [4]:
class Node:
  def __init__(self, word=None, op=None):
    self.word = word
    self.op = op
    self.ids = []
    self.size = 0
    self.left = None
    self.right = None

In [5]:
class BooleanRetrieval:
  def __init__(self):
    pass

  def buildTree(self, queryArray, InvertedIndex, optimize=False):
    operators = []
    operands = []
    totalDocuments = len(InvertedIndex.termDict)
    docIDs = InvertedIndex.termDict

    for token in queryArray:
      if token == "(":
        operators.append('(')
      elif token == ")":
        while operators and operators[-1] != '(':
          if optimize:
            self.reorderOperands(operands, operators, '&')
            self.reorderOperands(operands, operators, '/')

          self.updateTree(operands, operators, totalDocuments)

        operators.pop()
      elif token == "NOT":
        operators.append('~')
      elif token == "AND":
        while operators and operators[-1] == '~':
          self.updateTree(operands, operators, totalDocuments)

        operators.append('&')
      elif token == "OR":
        while operators and operators[-1] == '~':
          self.updateTree(operands, operators, totalDocuments)

        while operators and operators[-1] == '&':
          if optimize:
            self.reorderOperands(operands, operators, '&')

          self.updateTree(operands, operators, totalDocuments)

        operators.append('/')
      else:
        token = token.lower()
        node = Node(word=token)
        node.ids = docIDs.get(token, [])
        node.size = len(node.ids)
        operands.append(node)

        while operators and operators[-1] == '~':
          self.updateTree(operands, operators, totalDocuments)

    while operators:
      if optimize:
        self.reorderOperands(operands, operators, '&')
        self.reorderOperands(operands, operators, '/')

      self.updateTree(operands, operators, totalDocuments)

    return operands[-1]

  def printTree(self, root, prefix="", isLeft=True):
    if not root:
      return

    print(prefix + ("├── " if isLeft else "└── ") + (root.op if root.op else root.word))
    newPrefix = prefix + ("│   " if isLeft else "    ")

    if root.left or root.right:
      self.printTree(root.left, newPrefix, True)
      self.printTree(root.right, newPrefix, False)

  def parseQuery(self, query):
    result = []
    token = ""

    for c in query:
      if c in ('(', ')', ' '):
        if token:
          result.append(token)
          token = ""

        if c != ' ':
          result.append(c)
      else:
        token += c

    if token:
      result.append(token)

    return result

  def calculateTree(self, root, docSize):
    if not root:
      return []

    if root.op:
      left = self.calculateTree(root.left, docSize)
      right = self.calculateTree(root.right, docSize)

      if root.op == '&':
        return self.calculateAnd(left, right)
      elif root.op == '/':
        return self.calculateOr(left, right)
      elif root.op == '~':
        return self.calculateNot(right, docSize)
    else:
      return root.ids

  def reorderOperands(self, operands, operators, op):
    end = len(operators) - 1
    length = 1

    while operators[end] == op and end >= 0:
      end -= 1
      length += 1

    if length > 2:
      operands[-length:] = sorted(operands[-length:], key=lambda node: node.size, reverse=True)

  def updateTree(self, operands, operators, totalDocuments):
    op = operators.pop()

    right = operands.pop()
    node = Node(op=op)

    if op == '~':
      node.right = right
      node.size = totalDocuments - right.size
    else:
      left = operands.pop()
      node.left = left
      node.right = right

      if op == '&':
        node.size = min(left.size, right.size)
      else:
        node.size = min(totalDocuments, left.size + right.size)

    operands.append(node)

  def calculateAnd(self, left, right):
    return sorted(list(set(left) & set(right)))

  def calculateNot(self, right, docSize):
    return [i for i in range(docSize) if i not in right]

  def calculateOr(self, left, right):
    return sorted(list(set(left) | set(right)))


In [9]:
reader = CSVReader('../data/News.csv')
index = InvertedIndex()

start = time.time()
index.build(reader)
print(f'Building index took {time.time() - start} seconds')
print(f'Total Documents: {index.totalDocs}')
print(f'Total Terms: {index.totalTerm}')

OUTPUT_PATH = 'output.txt'
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
  for term, docIDs in index.termDict.items():
    f.write(f'{term}: {sorted(docIDs)}\n')

Finished building index
Building index took 23.811932802200317 seconds
Total Documents: 14343
Total Terms: 91016


In [ ]:
queries = [
  "NOT ((Indonesia AND NOT darurat) AND (covid OR covid19 OR (covid AND 19)))"
  "PEMERINTAH AND NOT covid AND Indonesia"
  "NOT (2020 OR 2021) AND (rabu OR Kamis OR minggu) AND bulutangkis"
  "((pemerintah AND tidak) OR (NOT militerisasi AND darutat)) AND segala AND (anak OR anaknya)"
  "(pembatasan OR pemberlakuan OR pengaturan) AND ((warga AND negara AND asing) OR WNA) AND ((masuk AND wilayah AND Indonesia) OR kebijakan) AND NOT pengecualian"
]

In [7]:
model = BooleanRetrieval()

for query in queries:
  start = time.time()
  qArray = model.parseQuery(query)
  root = model.buildTree(qArray, index)
  model.printTree(root)
  result = model.calculateTree(root, index.totalDocs)

  print(f'\nQuery took {time.time() - start} seconds')
  print(result)
  print(len(result), '\n\n')



Query took 0.006730079650878906 seconds
[944, 1237, 1402, 2670, 3027, 3032, 3388, 3621, 3622, 3671, 4500, 5186, 6255, 6342, 6343, 6631, 6667, 8399, 8803, 10838, 10842, 11190, 11525, 11895, 12246, 12847, 13554, 13572, 13906, 13911, 13986]
31


In [8]:
start = time.time()
rootOpt = model.buildTree(qArray, index, optimize=True)
model.printTree(rootOpt)
resultOpt = model.calculateTree(rootOpt, index.totalDocs)

print(f'\nQuery took {time.time() - start} seconds')
print(result)
print(len(result))

['(', 'pembatasan', 'OR', 'pemberlakuan', 'OR', 'pengaturan', ')', 'AND', '(', '(', 'warga', 'AND', 'negara', 'AND', 'asing', ')', 'OR', 'WNA', ')', 'AND', '(', '(', 'masuk', 'AND', 'wilayah', 'AND', 'Indonesia', ')', 'OR', 'kebijakan', ')', 'AND', 'NOT', 'pengecualian']

Query took 0.007503986358642578 seconds
[944, 1237, 1402, 2670, 3027, 3032, 3388, 3621, 3622, 3671, 4500, 5186, 6255, 6342, 6343, 6631, 6667, 8399, 8803, 10838, 10842, 11190, 11525, 11895, 12246, 12847, 13554, 13572, 13906, 13911, 13986]
31
